# INM707 Coursework
### Aaron Mir (Student Number: 160001207)
<img src="All_Tasks.png" alt="All_TasksOverview" width="700"/>  <img src="Task_1.png" alt="Task_1" width="700"/>

Environment: The agent is preparing for an upcoming mission in which they are to infiltrate an enemy stronghold to gather intelligence on a potential coup d'état. The enemy land is in the form of an NxN grid with each grid containing one stronghold of size N/2 x N or N x N/2 (starting on either side of the middle column (randomly decided)) and a wide river surrounding the entire enemy land. The stronghold has as many entrances from the mainland as 1/4*N and contains dangerous enemy combatants (at random positions) within the stronghold that move around per step (stochasticity). The land inside the stronghold has no transition probability associated with it (deterministic). The land surrounding the stronghold (the shore) is booby-trapped and covered in mist which means that there is both land which can kill the agent as well as normal land and a transition probability associated with each step (stochasticity) outside the stronghold. The amount of shore that has traps on it is 1/4 of the shore. The goal of the agent is to infiltrate the stronghold and gather the intelligence without being seen/killed by enemy combatants or booby traps.

The agent starts in a random cell which belongs to the land surrounding the stronghold.

Rewards: +10 for getting intelligence, -100 for getting killed by a combatant or booby trap, -1 for moving into a wall or water

Terminal States: agent moves to the intelligence, if agent moves to a trap and dies, if the agent moves to an enemy combatant.

N represents normal land [0], X represents a river [1], A represents the agent [2], W represents a wall [3], T represents a booby trap [4], E represents an enemy [5], I represents the intelligence [6]

More entrances the bigger the grid?

You don't need reward matrix and transition probability for q-learning

Task 1 should be visualisation and defining the stuff


In [21]:
import numpy as np
import matplotlib.pyplot as plt

In [24]:
size = 20
land = np.zeros((size, size))
land[0,:] = 1  
land[:,0] = 1
land[:,size-1] = 1
land[size-1,:] = 1  
print(land)

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [25]:
# try to make more efficient
choice = np.random.choice(('row', 'column')) # randomly deciding whether the stronghold will start 1 cell away from the middle row or middle column.
if choice == 'row':
    row_choice = np.random.choice((size//2, size//2+1, size//2-1))
    land[row_choice, 1:size-1] = 3
    if row_choice == size//2:
        choice_side = np.random.choice(('up', 'down'))
        print(choice_side)
        if choice_side == 'up':
            row_choice == size//2-1
        if choice_side == 'down':
            row_choice == size//2+1
    if row_choice == size//2-1:
        land[0:row_choice, 1] = 3
        land[0:row_choice, size-2] = 3
        land[0, 1:size-1] = 3
        traps = []
        for i in range(int(np.round(1/4*len(land[row_choice:size-2:,1:size-2]))))     # make as many traps as 1/4 of the shore NOT RIGHT
            trap = np.random.choice(np.setdiff1d(range(len(land[row_choice:size-2:,1:size-1])), traps))
            land[row_choice:size-2:,1:size-1][trap] = 4
            traps.append(trap)
    if row_choice == size//2+1:
        land[row_choice:size, 1] = 3
        land[row_choice:size, size-2] = 3
        land[size-1, 1:size-1] = 3
        traps = []
        for i in range(int(np.round(1/4*len(land[1:row_choice,1:size-1]))))     
            trap = np.random.choice(np.setdiff1d(range(len(land[1:row_choice,1:size-1])), traps))
            land[1:row_choice,1:size-1][trap] = 4
            traps.append(trap)
    entrances = []
    for i in range(int(np.round(1/4*len(land[row_choice,2:size-2])))+1):        # make as many entrances as 1/4 of the length of the front wall
        entrance = np.random.choice(np.setdiff1d(range(len(land[row_choice,2:size-1])), entrances))
        land[row_choice,2:size-1][entrance] = 0
        entrances.append(entrance_choice)
    
if choice == 'column':                  # above repeated for the column choice
    column_choice = np.random.choice((size//2, size//2+1, size//2-1))
    land[1:size-1, column_choice] = 3
    if column_choice == size//2:
        choice_side = np.random.choice(('left', 'right'))
        print(choice_side)
        if choice_side == 'left':
            column_choice == size//2-1
        if choice_side == 'right':
            column_choice == size//2+1
    if column_choice == size//2-1:
        land[1, 0:column_choice] = 3
        land[size-2, 0:column_choice] = 3
        land[1:size-1, 0] = 3
        traps = []
        for i in range(int(np.round(1/4*len(land[1:size-2, 1:column_choice]))))     
            trap = np.random.choice(np.setdiff1d(range(len(land[1:size-2, 1:column_choice])), traps))
            land[1:size-2, 1:column_choice][trap] = 4
            traps.append(trap)
    if column_choice == size//2+1:
        land[1, column_choice:size] = 3
        land[size-2, column_choice:size] = 3
        land[1:size-1, size-1] = 3
        traps = []
        for i in range(int(np.round(1/4*len(land[1:size-2, column_choice:size-2]))))     
            trap = np.random.choice(np.setdiff1d(range(len(land[1:size-2, column_choice:size-2])), traps))
            land[1:size-2, column_choice:size-2][trap] = 4
            traps.append(trap)
    entrances = []
    for i in range(int(np.round(1/4*len(land[2:size-1, column_choice])))+1):
        entrance = np.random.choice(np.setdiff1d(range(len(land[2:size-1, column_choice])), entrances))
        land[2:size-1, column_choice][entrance] = 0
        entrances.append(entrance_choice)
print(land)    

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 3. 3. 3. 0. 0. 3. 3. 0. 3. 3. 3. 3. 3. 3. 3. 3. 0. 0. 1.]
 [1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 3. 1.]
 [1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 3. 1.]
 [1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 3. 1.]
 [1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [31]:
print(land)

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 3. 3. 3. 0. 0. 3. 3. 0. 3. 3. 3. 3. 3. 3. 3. 3. 0. 0. 1.]
 [1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 3. 1.]
 [1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 3. 1.]
 [1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 3. 1.]
 [1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [36]:
np.size(land[1:row_choice,1:size-1])

180

In [ ]:
traps = []
for i in range(int(np.round(1/4*np.size(land[1:row_choice,1:size-1]))))     
    trap = np.random.choice(np.setdiff1d(range(len(land[1:row_choice,1:size-1])), traps))
    land[1:row_choice,1:size-1][trap] = 4
    traps.append(trap)

In [200]:
np.random.choice(land[2:size-1, 7], 3)

array([3., 0., 3.])

In [ ]:
class Stronghold():
    def __init__(self, size):
        self.size = size
        self.actions = {0: 'Up', 1: 'Down', 2: 'Left', 3: 'Right'}
        self.land = self.env_gen()

    def env_gen(self):
        land = np.zeros((self.size, self.size))
        land[0,:] = 1
        land[:,0] = 1
        land[self.size-1,:] = 1
        land[:,self.size-1] = 1
        return land
    
    def step(self, action):
        
        return next_state, reward, done
    
    def reset():
        pass

    def render(): # displays
        pass
''' Don't need this for Q-learning
    def fill_transition_probability(self): # p.106
        
        return P
    
    def fill_reward_matrix(self):
        
        return R
'''

In [ ]:
class Policies():
    def __init__(self):
        pass
    
    def random_policy(self):
        pass

In [ ]:
env = Stronghold(10)